<a href="https://colab.research.google.com/github/chrisdantas/chrisdantas.github.io/blob/main/C%C3%B3pia_FINAL_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será lançada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho no Instagram, LinkedIn e outros canais da Alura, pois eles costumam anunciar novos cursos e imersões por lá.
*   **Verificar a página de cursos de IA da Alura:** A Alura possui uma página dedicada aos cursos de Inteligência Artificial. Confira periodicamente para ver se há alguma novidade sobre a Imersão IA com Google Gemini.
*   **Entrar na comunidade da Alura:** Participar da comunidade da Alura é uma ótima maneira de ficar por dentro dos próximos eventos e lançamentos.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Inserir a tool de busca do Google ###############################################
    config={"tools":[{"google_search":{}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura ocorreu de 12 a 16 de maio de 2025. As inscrições para o curso, que é gratuito, encerraram em 11 de maio de 2025.

In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Buscador #################################################
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search) para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações negativas, é possível que ele não seja tão relevante para assim e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca notícias sobre o tópico indicado no Google",
        tools=[google_search]
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados


In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de lançamentos mais recentes e relevantes no buscador, você deve:
        - Usar a ferramenta de busca do google (google search) para criar um plano sobre quais são ospontos mais relevantes que poderíamos abordar em um post sobre cada um
        deles e encontrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas e retornar esse tema, seus pontos mais relevantes e um plano com
        os assuntos a serem abordados nopost que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de diitar o tópico? 🤔")
else:
  print(f"Maravilha! Você quer criar o post sobre novidades em '{topico}'?")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n-- Resultados do Buscador --")
  display(to_markdown(lancamentos_buscados))
  print("------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n-- Resultados do Planejador --")
  display(to_markdown(plano_de_post))
  print("------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n-- Resultados do Redator --")
  display(to_markdown(rascunho_de_post))
  print("------------------")

  post_final = agente_revisor(topico, rascunho_gerado)
  print("\n-- Resultados do Revisor --")
  display(to_markdown(post_final))
  print("------------------")


🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: agentes de IA
Maravilha! Você quer criar o post sobre novidades em 'agentes de IA'?

-- Resultados do Buscador --


> Aqui estão algumas notícias sobre lançamentos recentes e relevantes de agentes de IA:
> 
> 
> Com base nas notícias mais recentes, aqui estão 5 lançamentos e tendências notáveis no campo de agentes de IA:
> 
> 1.  **Adoção e Integração Crescente de Agentes de IA:** Empresas como IBM, Anthropic, Amazon, Google e OpenAI estão anunciando atualizações e lançamentos que visam revolucionar a inteligência artificial, com foco na integração avançada de agentes. A IBM, por exemplo, anunciou atualizações no watsonx Orchestrate, oferecendo soluções para integrar, customizar e implementar agentes em áreas como RH, vendas e compras.
> 2.  **Agentes de IA Autônomos Transformando o Comércio:** Um relatório da Visa e do Institute for the Future (IFTF) indica que os agentes de IA têm o potencial de revolucionar a forma como os indivíduos se envolvem com finanças. O relatório aponta que uma grande porcentagem de brasileiros demonstra interesse em assistentes pessoais com tecnologia de IA.
> 3.  **Expansão da Aplicação de Agentes de IA em Empresas:** O iFood, maior aplicativo de delivery do Brasil, está implementando agentes de IA da Salesforce para automatizar tarefas internas, com planos de expandir o uso desses agentes significativamente. Esses agentes estão sendo utilizados para otimizar o atendimento ao cliente e agilizar processos de vendas.
> 4.  **Agentes de IA Autônomos e a Transformação da Criação de Conteúdo Digital:** Ferramentas de IA prometem alterar a forma como agências de publicidade, designers e criadores de conteúdo trabalham. A automação na criação de conteúdo é uma área que será profundamente transformada, com agentes autônomos capazes de acessar sistemas internos e dados proprietários, possibilitando automação avançada em diversas áreas.
> 5.  **Avanços Tecnológicos e a Evolução dos Agentes de IA:** A inteligência artificial está atingindo um novo patamar, com agentes de IA combinando maior poder de síntese de dados e capacidades aprimoradas de tomada de decisão e interação com múltiplas ferramentas. Empresas como a GIGABYTE estão apresentando inovações orientadas por IA, incluindo PCs com IA e plataformas avançadas, demonstrando como os ecossistemas estão definindo a próxima era da computação inteligente e personalizada.
> 
> 


------------------

-- Resultados do Planejador --


> Ok, com base nas notícias sobre lançamentos e tendências de agentes de IA, vou criar um plano para posts em redes sociais. Primeiro, farei algumas pesquisas para aprofundar cada um dos temas.
> 
> 
> Após analisar as informações, o tema mais relevante para um post de mídia social é:
> 
> **Agentes de IA Autônomos Transformando o Comércio e as Finanças Pessoais**
> 
> **Pontos Relevantes:**
> 
> *   **Relatório Visa e IFTF:** Agentes de IA têm o potencial de revolucionar a forma como as pessoas lidam com finanças.
> *   **Interesse do consumidor:** Grande porcentagem de brasileiros demonstra interesse em assistentes pessoais com tecnologia de IA.
> *   **Confiança e Segurança:** A confiança e a segurança são cruciais na adoção de agentes de IA no comércio.
> *   **Personalização:** Agentes de IA podem oferecer experiências de compra e gestão financeira mais personalizadas.
> *   **Aplicações Práticas:** Os agentes de IA podem auxiliar em diversas tarefas financeiras, desde a negociação de preços até a compra de produtos.
> 
> **Plano para o Post:**
> 
> 1.  **Título:** "Agentes de IA: O Futuro das Suas Finanças Pessoais Chegou?" (ou algo similar chamativo).
> 2.  **Introdução:**
>     *   Mencionar o relatório da Visa e do Institute for the Future (IFTF) que destaca o potencial transformador dos agentes de IA no comércio e finanças.
>     *   Destacar o crescente interesse dos consumidores brasileiros em assistentes pessoais com IA.
> 3.  **Desenvolvimento:**
>     *   Explicar como os agentes de IA podem revolucionar a forma como as pessoas se envolvem com finanças.
>     *   Exemplos de tarefas que agentes de IA podem realizar:
>         *   Gerenciar orçamentos
>         *   Negociar preços
>         *   Realizar compras
>         *   Investimentos personalizados
>     *   Discutir a importância da confiança e segurança na adoção de agentes de IA. Mencionar como a Visa está trabalhando para garantir a segurança nas transações com IA.
> 4.  **Exemplos e Casos de Uso:**
>     *   Mencionar como outras empresas estão implementando IA para melhorar a experiência do cliente no setor financeiro (se houver exemplos relevantes encontrados nas buscas).
> 5.  **Chamada para Ação:**
>     *   Incentivar os seguidores a compartilhar suas opiniões sobre o uso de agentes de IA em suas finanças pessoais.
>     *   Perguntar se eles usariam um assistente pessoal com IA para gerenciar seu dinheiro.
> 6.  **Hashtags:**
>     *   #IA #InteligenciaArtificial #AgentesDeIA #FinançasPessoais #Visa #Inovação #Tecnologia #FuturoDoComercio
> 
> **Justificativa para a escolha:**
> 
> Este tema é relevante porque combina inovação tecnológica (agentes de IA) com um aspecto prático e de interesse geral (finanças pessoais). O relatório da Visa/IFTF adiciona credibilidade e dados interessantes para o post. Além disso, o tema permite gerar discussões e engajamento com o público, incentivando-os a refletir sobre o futuro das finanças pessoais e o papel da IA nesse contexto.
> 


------------------

-- Resultados do Redator --


> 🚨 Agentes de IA: O Futuro das Suas Finanças Pessoais Chegou? 🤖💰
> 
> Já imaginou ter um assistente pessoal turbinado por Inteligência Artificial cuidando do seu dinheiro? 🤯 Um estudo recente da Visa e do Institute for the Future (IFTF) revelou o potencial INACREDITÁVEL dos agentes de IA para revolucionar a forma como lidamos com nossas finanças! 🚀
> 
> E não é só isso: os brasileiros estão SUPER interessados em ter assistentes pessoais com IA. 🤩 Mas, calma! A confiança e a segurança são FUNDAMENTAIS nessa história. Afinal, estamos falando do seu dinheiro, né? 😉
> 
> ✨ **O que esses agentes de IA podem fazer por você?** ✨
> 
> *   Gerenciar seu orçamento 📊
> *   Negociar preços (quem não ama um desconto?) 🤑
> *   Realizar compras inteligentes 🛍️
> *   Criar investimentos personalizados 📈
> 
> A Alura está de olho nessa revolução da IA e como ela pode transformar o mundo! 👀 Queremos saber: você confiaria em um agente de IA para cuidar das suas finanças? 🤔 Conta pra gente nos comentários! 👇
> 
> #IA #InteligenciaArtificial #FinançasPessoais #Inovação #Alura
> 


------------------

-- Resultados do Revisor --


> Por favor, forneça o rascunho do post de Instagram sobre agentes de IA para que eu possa revisá-lo.
> 


------------------
